In [37]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import os
import json
import numpy as np 
import pandas as pd 
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
#from jsonpath_ng import parse
from enum import Enum
from typing import List
%matplotlib inline 

Source to Target Mapping




In [74]:
data = pd.read_csv("flightsJoin.csv")
data


<ipython-input-74-a63466f53bca>:1: DtypeWarning: Columns (2,7,15,16,17,18,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("flightsJoin.csv")


,Unnamed: 0,Airline,Airline ID,Source Airport,Source Airport ID,Destination Airport,Destination Airport ID,Codeshare,Stops,Equipment,...,Name_y,City,Country_y,IATA,Latitude,Longitude,Altitude,Timezone,DST,Timezone.1
0,0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32922,32922,GS,690,CGO,3375,TAO,3390,NaN,0,E90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32923,32923,GS,690,CGO,3375,TSN,3368,NaN,0,E90 320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32924,32924,GS,690,CGO,3375,TSN,3368,NaN,0,E90 320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32925,32925,GS,690,CGO,3375,URC,3399,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df = pd.read_csv('flightsJoin.csv', low_memory=False)
for dtype in df.dtypes.items():
  print(dtype)

('Unnamed: 0', dtype('int64'))
('Airline', dtype('O'))
('Airline ID', dtype('O'))
('Source Airport', dtype('O'))
('Source Airport ID', dtype('O'))
('Destination Airport', dtype('O'))
('Destination Airport ID', dtype('O'))
('Codeshare', dtype('O'))
('Stops', dtype('int64'))
('Equipment', dtype('O'))
('Name_x', dtype('O'))
('ICAO', dtype('O'))
('Callsign', dtype('O'))
('Country_x', dtype('O'))
('Active', dtype('O'))
('Name_y', dtype('O'))
('City', dtype('O'))
('Country_y', dtype('O'))
('IATA', dtype('O'))
('Latitude', dtype('float64'))
('Longitude', dtype('float64'))
('Altitude', dtype('float64'))
('Timezone', dtype('float64'))
('DST', dtype('O'))
('Timezone.1', dtype('O'))


Absract base class

In [41]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

Transform Operations

In [42]:
class TransformMask(Enum):
    # add here
    CLEAN_STRING = ".strip().lower()" 
    CAPITAL_LETTER = ".strip().lower().title()"
    REMOVE_NULL = "apps.dropna(inplace=True)"

Database

In [64]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }

        self.add_source(1, "Airline", "$.Airline", "str", True)
        self.add_source(2, "Source Airport", "$.Source Airport", "str", True)
        self.add_source(3, "Destination Airport", "$.Destination Airport", "str", True)
        self.add_source(4, "Stops", "$.Stops", "int", True)
        self.add_source(5, "Name_x", "$.Name_x", "str", True)
        self.add_source(6, "ICAO", "$.ICAO", "str", True)
        self.add_source(7, "Country_x", "$.Country_x", "str", True)
        self.add_source(8, "Active", "$.Active", "str", True)
        self.add_source(9, "Name_y", "$.Name_y", "str", True)
        self.add_source(10, "City", "$.City", "str", True)
        self.add_source(11, "Country_y", "$.Country_y", "str", True)
        self.add_source(12, "IATA", "$.IATA", "str", True)

        self.add_destination(1, "Airline", "$.Airline", "str", "n/a")
        self.add_destination(2, "Source Airport", "$.Source_Airport", "str", "n/a")
        self.add_destination(3, "Destination Airport", "$.Destination_Airport", "str", "n/a")
        self.add_destination(4, "Stops", "$.Stops", "int", "0")
        self.add_destination(5, "Name_x", "$.Airline_Name", "str", "n/a")
        self.add_destination(6, "ICAO", "$.ICAO", "str", "n/a")
        self.add_destination(7, "Country_x", "$.Airline_Country", "str", "n/a")
        self.add_destination(8, "Active", "$.Active", "str", "n/a")
        self.add_destination(9, "Name_y", "$.Airport_Name", "str", "n/a")
        self.add_destination(10, "City", "$.Airport_City", "str", "n/a")
        self.add_destination(11, "Country_y", "$.Airport_Country", "str", "n/a")
        self.add_destination(12, "IATA", "$.IATA", "str", "n/a")

        self.add_transform(1, "REMOVE_NULL")
        self.add_transform(2, "REMOVE_NULL")
        self.add_transform(3, "REMOVE_NULL")
        self.add_transform(4, "REMOVE_NULL")
        self.add_transform(5, "REMOVE_NULL")
        self.add_transform(6, "REMOVE_NULL")
        self.add_transform(7, "REMOVE_NULL")
        self.add_transform(8, "REMOVE_NULL")
        self.add_transform(9, "REMOVE_NULL")
        self.add_transform(10, "REMOVE_NULL")
        self.add_transform(11, "REMOVE_NULL")
        self.add_transform(12, "REMOVE_NULL")

        self.add_mapping(1, 1, 1, 1)
        self.add_mapping(2, 2, 2, 2)
        self.add_mapping(3, 3, 3, 3)
        self.add_mapping(4, 4, 4, 4)
        self.add_mapping(5, 5, 5, 5)
        self.add_mapping(6, 6, 6, 6)
        self.add_mapping(7, 7, 7, 7)
        self.add_mapping(8, 8, 8, 8)
        self.add_mapping(9, 9, 9, 9)
        self.add_mapping(10, 10, 10, 10)
        self.add_mapping(11, 11, 11, 11)
        self.add_mapping(12, 12, 12, 12)

        

    def add_source(self, id, name, mapping, typo, is_required):
      self.db["source"].append({
                    "id": id,
                    "source_field_name": name,
                    "source_field_mapping": mapping,
                    "source_field_type": typo, # use python types
                    "is_required": is_required,
                     })

    def add_destination(self, id, name, mapping, typo, default):
      self.db["destination"].append({
                    "id": id,
                    "destination_field_name": name,
                    "destination_field_mapping": mapping,
                    "destination_field_type": typo,
                    "default_value": default,
                     })
      
     

    def add_transform(self, id, transform_mask):
      self.db["transform"].append({
                  "id": id,
                  "transform_mask": transform_mask,
                    })

    def add_mapping(self, id, mapping_source, mapping_destination, mapping_transform):
      self.db["mapping"].append({
                  "id": id,
                  "mapping_source": mapping_source,
                  "mapping_destination": mapping_destination,
                  "mapping_transform": mapping_transform,
                    })

    # built-in function that creates and returns a property object
    # get data by: get_data_source_target_mapping.get(dict_key)
    
    @property 
    def get_data_source_target_mapping(self):
        return self.db


Source

In [65]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)
        Interface.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

Target

In [66]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)
        Interface.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

Transform

In [67]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)
        Interface.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

Mapping

In [68]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)
        Interface.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

Convert to Json

In [90]:
# import csv 
# import json 

# def csv_to_json(csvFilePath, jsonFilePath):
#     jsonArray = []
      
#     #read csv file
#     with open(csvFilePath, encoding='utf-8') as csvf: 
#         #load csv file data using csv library's dictionary reader
#         csvReader = csv.DictReader(csvf) 

#         #convert each csv row into python dict
#         for row in csvReader: 
#             #add this python dict to json array
#             jsonArray.append(row)
  
#     #convert python jsonArray to JSON String and write to file
#     with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
#         jsonString = json.dumps(jsonArray, indent=4)
#         jsonf.write(jsonString)
          
# csvFilePath = r'flightsJoin.csv'
# jsonFilePath = r'flights.json'
# csv_to_json(csvFilePath, jsonFilePath)


# import csv
# import json


# # Function to convert a CSV to JSON
# # Takes the file paths as arguments
# def make_json(csvFilePath, jsonFilePath):
	
# 	# create a dictionary
# 	data = {}
	
# 	# Open a csv reader called DictReader
# 	with open(csvFilePath, encoding='utf-8') as csvf:
# 		csvReader = csv.DictReader(csvf)
		
# 		# Convert each row into a dictionary
# 		# and add it to data
# 		for rows in csvReader:
			
# 			# Assuming a column named 'No' to
# 			# be the primary key
# 			key = rows['Airline ID']
# 			data[key] = rows

# 	# Open a json writer, and use the json.dumps()
# 	# function to dump data
# 	with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
# 		jsonf.write(json.dumps(data, indent=4))
		
# Driver Code

# Decide the two file paths according to your
# computer system
# csvFilePath = r'flightsJoin.csv'
# jsonFilePath = r'Flights1.json'

# # Call the make_json function
# make_json(csvFilePath, jsonFilePath)

# with open("Flights1.json", "r") as file:
#     data = json.loads(json.load(json.dumps(file)))
#     print(data)
# data[:10]

In [70]:
#in order to be able to search in the data

class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

Combine

In [71]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [97]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

AttributeError: ignored

In [ ]:
pd.DataFrame(transformed_data)

In [ ]:
#pd.DataFrame(transformed_data).groupby("job_title").mean()